In [1]:
import itertools

In [2]:
import numpy as np

In [3]:
def heuristic(C):
    S = np.zeros(C.shape)
    for i, j in itertools.product(*[range(d) for d in C.shape]):
        if C[i,j].round(2) != 0:
            S[i,j] = -np.infty
            continue
        mr, mc = 2*[np.infty]
        for i1 in range(C.shape[0]):
            if i1 == i: continue
            mc = min(mc, C[i1, j])
        for j1 in range(C.shape[1]):
            if j1 == j: continue
            mr = min(mr, C[i, j1])
        S[i,j] = mr + mc
    return S

In [4]:
def row_transform(C):
    mins = np.min(C, axis=1).reshape((-1,1))
    return C - mins, mins

def col_transform(C):
    mins = np.min(C, axis=0)
    return C - mins, mins

In [5]:
N=6
C = np.array([
    [7+5*np.abs(np.sin(i+1)*np.cos(1+j)) for j in range(N)]
    for i in range(N)
])
C = C.round(2)
C

array([[  9.27,   8.75,  11.17,   9.75,   8.19,  11.04],
       [  9.46,   8.89,  11.5 ,   9.97,   8.29,  11.37],
       [  7.38,   7.29,   7.7 ,   7.46,   7.2 ,   7.68],
       [  9.04,   8.57,  10.75,   9.47,   8.07,  10.63],
       [  9.59,   9.  ,  11.75,  10.13,   8.36,  11.6 ],
       [  7.75,   7.58,   8.38,   7.91,   7.4 ,   8.34]])

In [6]:
for i in range(N):
    C[i, i] = np.infty
C

array([[   inf,   8.75,  11.17,   9.75,   8.19,  11.04],
       [  9.46,    inf,  11.5 ,   9.97,   8.29,  11.37],
       [  7.38,   7.29,    inf,   7.46,   7.2 ,   7.68],
       [  9.04,   8.57,  10.75,    inf,   8.07,  10.63],
       [  9.59,   9.  ,  11.75,  10.13,    inf,  11.6 ],
       [  7.75,   7.58,   8.38,   7.91,   7.4 ,    inf]])

In [7]:
C2, C2mr = row_transform(C)
C2 = C2.round(2)
C2

array([[  inf,  0.56,  2.98,  1.56,  0.  ,  2.85],
       [ 1.17,   inf,  3.21,  1.68,  0.  ,  3.08],
       [ 0.18,  0.09,   inf,  0.26,  0.  ,  0.48],
       [ 0.97,  0.5 ,  2.68,   inf,  0.  ,  2.56],
       [ 0.59,  0.  ,  2.75,  1.13,   inf,  2.6 ],
       [ 0.35,  0.18,  0.98,  0.51,  0.  ,   inf]])

In [8]:
C2mr

array([[ 8.19],
       [ 8.29],
       [ 7.2 ],
       [ 8.07],
       [ 9.  ],
       [ 7.4 ]])

In [9]:
C2, C2mc = col_transform(C2)
C2 = C2.round(2)
C2

array([[  inf,  0.56,  2.  ,  1.3 ,  0.  ,  2.37],
       [ 0.99,   inf,  2.23,  1.42,  0.  ,  2.6 ],
       [ 0.  ,  0.09,   inf,  0.  ,  0.  ,  0.  ],
       [ 0.79,  0.5 ,  1.7 ,   inf,  0.  ,  2.08],
       [ 0.41,  0.  ,  1.77,  0.87,   inf,  2.12],
       [ 0.17,  0.18,  0.  ,  0.25,  0.  ,   inf]])

In [10]:
C2mc

array([ 0.18,  0.  ,  0.98,  0.26,  0.  ,  0.48])

In [11]:
l2r, l2c = C2mr.sum().round(2), C2mc.sum().round(2)
l2 = l2r + l2c
l2r, l2c, l2

(48.149999999999999, 1.8999999999999999, 50.049999999999997)

In [12]:
H2 = heuristic(C2)
H2

array([[ -inf,  -inf,  -inf,  -inf,  0.56,  -inf],
       [ -inf,  -inf,  -inf,  -inf,  0.99,  -inf],
       [ 0.17,  -inf,  -inf,  0.25,  0.  ,  2.08],
       [ -inf,  -inf,  -inf,  -inf,  0.5 ,  -inf],
       [ -inf,  0.5 ,  -inf,  -inf,  -inf,  -inf],
       [ -inf,  -inf,  1.7 ,  -inf,  0.  ,  -inf]])

In [13]:
B1u = 8.19+9+9.97+10.63+8.38+7.38
B1u, round(B1u, 2)

(53.550000000000004, 53.55)

In [14]:
C2

array([[  inf,  0.56,  2.  ,  1.3 ,  0.  ,  2.37],
       [ 0.99,   inf,  2.23,  1.42,  0.  ,  2.6 ],
       [ 0.  ,  0.09,   inf,  0.  ,  0.  ,  0.  ],
       [ 0.79,  0.5 ,  1.7 ,   inf,  0.  ,  2.08],
       [ 0.41,  0.  ,  1.77,  0.87,   inf,  2.12],
       [ 0.17,  0.18,  0.  ,  0.25,  0.  ,   inf]])

In [15]:
round(50.05+2.08, 2)

52.13

In [16]:
C_2_1 = C2[np.array([1,2,4,5,6])-1, :]
C_2_1 = C_2_1[:,np.array([1,2,3,4,5])-1]
C_2_1[-1, 3-1] = np.infty
C_2_1

array([[  inf,  0.56,  2.  ,  1.3 ,  0.  ],
       [ 0.99,   inf,  2.23,  1.42,  0.  ],
       [ 0.79,  0.5 ,  1.7 ,   inf,  0.  ],
       [ 0.41,  0.  ,  1.77,  0.87,   inf],
       [ 0.17,  0.18,   inf,  0.25,  0.  ]])

In [17]:
C3, C3_mr = row_transform(C_2_1)
C3

array([[  inf,  0.56,  2.  ,  1.3 ,  0.  ],
       [ 0.99,   inf,  2.23,  1.42,  0.  ],
       [ 0.79,  0.5 ,  1.7 ,   inf,  0.  ],
       [ 0.41,  0.  ,  1.77,  0.87,   inf],
       [ 0.17,  0.18,   inf,  0.25,  0.  ]])

In [18]:
C3_mr

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [19]:
C3, C3_mc = col_transform(C3)
C3

array([[  inf,  0.56,  0.3 ,  1.05,  0.  ],
       [ 0.82,   inf,  0.53,  1.17,  0.  ],
       [ 0.62,  0.5 ,  0.  ,   inf,  0.  ],
       [ 0.24,  0.  ,  0.07,  0.62,   inf],
       [ 0.  ,  0.18,   inf,  0.  ,  0.  ]])

In [20]:
C3_mc

array([ 0.17,  0.  ,  1.7 ,  0.25,  0.  ])

In [21]:
l3 = C3_mr.sum() + C3_mc.sum()
l3.round(2)

2.1200000000000001

In [22]:
round(l2 + l3, 2)

52.170000000000002

In [23]:
H3 = heuristic(C3)
H3

array([[ -inf,  -inf,  -inf,  -inf,  0.3 ],
       [ -inf,  -inf,  -inf,  -inf,  0.53],
       [ -inf,  -inf,  0.07,  -inf,  0.  ],
       [ -inf,  0.25,  -inf,  -inf,  -inf],
       [ 0.24,  -inf,  -inf,  0.62,  0.  ]])

In [29]:
52.17+.53

52.7

In [34]:
C4 = C3[[0,2,3,4,], :][:, :-1]
C4[2, 1] = np.infty
C4

array([[  inf,  0.56,  0.3 ,  1.05],
       [ 0.62,  0.5 ,  0.  ,   inf],
       [ 0.24,   inf,  0.07,  0.62],
       [ 0.  ,  0.18,   inf,  0.  ]])

In [35]:
C4, C4mr = row_transform(C4)
print(C4mr)
C4

[[ 0.3 ]
 [ 0.  ]
 [ 0.07]
 [ 0.  ]]


array([[  inf,  0.26,  0.  ,  0.75],
       [ 0.62,  0.5 ,  0.  ,   inf],
       [ 0.17,   inf,  0.  ,  0.55],
       [ 0.  ,  0.18,   inf,  0.  ]])

In [39]:
C4, C4mc = col_transform(C4)
print(C4mc)
C4

[ 0.    0.18  0.    0.  ]


array([[  inf,  0.08,  0.  ,  0.75],
       [ 0.62,  0.32,  0.  ,   inf],
       [ 0.17,   inf,  0.  ,  0.55],
       [ 0.  ,  0.  ,   inf,  0.  ]])

In [40]:
H4 = heuristic(C4)
H4

array([[ -inf,  -inf,  0.08,  -inf],
       [ -inf,  -inf,  0.32,  -inf],
       [ -inf,  -inf,  0.17,  -inf],
       [ 0.17,  0.08,  -inf,  0.55]])

In [42]:
L4 = C4mr.sum() + C4mc.sum()
L4

0.55000000000000004

In [44]:
L4 + l3 + l2

52.719999999999999

In [46]:
round(52.72+.55, 2)

53.27

In [47]:
C5 = C4[:-1,:][:,:-1]
C5

array([[  inf,  0.08,  0.  ],
       [ 0.62,  0.32,  0.  ],
       [ 0.17,   inf,  0.  ]])

In [48]:
C5, C5mc = col_transform(C5)
C5

array([[  inf,  0.  ,  0.  ],
       [ 0.45,  0.24,  0.  ],
       [ 0.  ,   inf,  0.  ]])

In [49]:
C5mc.sum(), C5mc

(0.24999999999999992, array([ 0.17,  0.08,  0.  ]))

In [51]:
L5 = C5mc.sum()
round(L5 + L4 + l3 + l2, 2)

52.969999999999999

In [53]:
H5 = heuristic(C5)
H5

array([[ -inf,  0.24,  0.  ],
       [ -inf,  -inf,  0.24],
       [ 0.45,  -inf,  0.  ]])

In [54]:
52.97 + .45

53.42

In [55]:
52.97+.32

53.29

In [56]:
C7 = C4.copy()
C7

array([[  inf,  0.08,  0.  ,  0.75],
       [ 0.62,  0.32,  0.  ,   inf],
       [ 0.17,   inf,  0.  ,  0.55],
       [ 0.  ,  0.  ,   inf,  0.  ]])